# FRA 503: Deep Reinforment Learning
HW 2 Cartpole

Napat Aeimwiratchai 65340500020
Phattarawat Kadrum 65340500074

**Learning Objectives:**

1. Understand how a **reinforcement learning agent learns** (i.e., evaluates and improves its policy) in an environment where the true dynamic model is unknown.

2. Gain insight into different **reinforcement learning algorithms**, including Monte Carlo methods, the SARSA algorithm, Q-learning, and Double Q-learning. Analyze their strengths and weaknesses.

3. Explore approaches to implementing reinforcement learning in **real-world scenarios where the state and action spaces are continuous.**

### **Part 1 : Setting up `Cart-Pole` Agent**

#### **1. RL Base class**

- **Constructor (__init__) to initialize the following parameters:**

    - **Control type:** Enumeration of RL algorithms used for decision-making (i.e. Monte Carlo, Temporal Difference, Q-learning, or Double Q-learning).

    - **Number of actions:** The total number of discrete actions available to the agent.

    - **Action range:** The minimum and maximum values defining the range of possible actions.

    - **Discretize state weight:** Weighting factor applied when discretizing the state space for learning.

    - **Learning rate:** Determines how quickly the model updates based on new information.

    - **Initial epsilon:** The starting probability of taking a random action in an ε-greedy policy.

    - **Epsilon decay rate:** The rate at which epsilon decreases over time to favor exploitation over exploration.

    - **Final epsilon:** The lowest value epsilon can reach, ensuring some level of exploration remains.

    - **Discount factor:** A coefficient (γ) that determines the importance of future rewards in decision-making.

- **Core Functions**

    - **get_discretize_action():** Returns a discrete action based on the current policy.

    - **mapping_action():** Converts a discrete action back into a continuous action within the defined action range.

    - **discretize_state(): Discretizes the state based on observation weights by divide into period.**

    - **decay_epsilon():** Decreases epsilon over time and returns the updated value.

#### **2. Algorithm folder**

Implement the algorithm

- **`Monte Carlo class`**

```py
     def update(self):
        """
        Update Q-values using reverse-order, first-visit Monte Carlo.
        """
        G = 0  # Initialize return
        visited_pairs = set()

        # Traverse episode in reverse
        for t in reversed(range(len(self.reward_hist))):
            state = self.obs_hist[t]
            action = self.action_hist[t]
            reward = self.reward_hist[t]

            G = self.discount_factor * G + reward  # Incremental return

            # First-visit MC check
            if (state, action) not in visited_pairs:
                visited_pairs.add((state, action))
                self.n_values[state][action] += 1
                alpha = 1 / self.n_values[state][action]  # Incremental mean
                self.q_values[state][action] += alpha * (G - self.q_values[state][action])

        # Clear episode history
        self.obs_hist.clear()
        self.action_hist.clear()
        self.reward_hist.clear()

        # Decay epsilon after episode
        self.decay_epsilon()
```

- **`SARSA class`**

```py
        def update(self , state, action_idx, reward, next_state, next_action_idx, done):
        """
        Update Q-values using SARSA .
        """

        self.q_values[state][action_idx] += self.lr * (
            reward + self.discount_factor * self.q_values[next_state][next_action_idx] - self.q_values[stat][action_idx]
        )
        
        # Move to next state-action pair for the next step in the episode
        state = next_state
        action_idx = next_action_idx
```

- **`Q-Learning Class`**

```py
     def update(self , state, action_idx, reward, next_state):
        """
        Update Q-values using Q-Learning.
        """
        self.q_values[state][action_idx] += self.lr * (
            reward + self.discount_factor * np.max(self.q_values[next_state]) - self.q_values[state][action_idx]
        )
```

- **`Double Q-Learning Class1`**

```py
            
    def update(self , state, action_idx, reward, next_state):
        """
        Update Q-values using Double Q-Learning.
        """

        if np.random.rand() < 0.5:
            self.qa_values[state][action_idx] += self.lr * (
                reward + self.discount_factor * self.qb_values[next_state][np.argmax(self.qa_values[next_state])]
                - self.qa_values[state][action_idx]
            )
        else:
            self.qb_values[state][action_idx] += self.lr * (
                reward + self.discount_factor * self.qa_values[next_state][np.argmax(self.qb_values[next_state])]
                - self.qb_values[state][action_idx]
            )
        self.q_values[state][action_idx] = self.qa_values[state][action_idx] + self.qb_values[state][action_idx]
        
```


### Part 2 : Training & Playing to stabilize `Cart-Pole` Agent

#### **A. Discrete state and action**

For convert continuous to discrete state and action of `cartpole` we need to set `discretize_state_weight` (pose_cart, pose_pole, vel_cart, vel_pole) for divide range of position, velocity to discrete step and `num_of_action` is for divide range of action to discrete step.

##### **Experiment Config (State, Action)**  
 Experiment for testing how number of actions and discretize_state_weights effect to the `cartpole` task.


 ```py
 action_of_action = xx  # Number of available discrete action
 discretize_state_weight = [xx, xx, xx, xx] #  [pose_cart:int, pose_pole:int, vel_cart:int, vel_pole:int]

 ```

#### **B. Hyperparameter adjusting**

For observe how hyperparameter effect to learning trend of `CartPole` agent, we try to adjust of 3 parameters (`Learning Rate`, `Epsilon Decay`, `Discount Factor`).

##### **Experiment Config (Hyperparameter)**  
This experiment for testing how algorithm's hyperparameter effect to result of `CartPole` task.

```py
    learning_rate = xx 
    n_episodes = xx     # Number of episode for Agent's train
    start_epsilon = xx  
    epsilon_decay = xx  # Reduce the exploration over time
    final_epsilon = xx  
    discount = xx       # Discount factor
```

### Part 3 : Evaluate `Cart-Pole` Agent performance


### Reward Graph tensorboard 

#### Hyperparameter

##### Learning rate
In this experiment we use 2 value of learning rate (0.1 and 0.5)

SARSA : 
$$q_{t+1}(S_t, A_t) \leftarrow q_t(S_t, A_t) + \alpha_t \left[ R_{t+1} + \gamma q(S_{t+1}, A_{t+1}) - q_t(S_t, A_t) \right]
$$

Q_Learning :
$$
q_{t+1}(S_t, A_t) \leftarrow q_t(S_t, A_t) + \alpha_t \left[ R_{t+1} + \gamma \max_{a'} q_t(S_{t+1}, a') - q_t(S_t, A_t) \right]
$$

Double_Q_Learning :
$$
q_{t+1}(S_t, A_t) \leftarrow q_t(S_t, A_t) + \alpha_t \left[ R_{t+1} + \gamma \max_{a'} q'_t(S_{t+1}, a') - q_t(S_t, A_t) \right]
$$

$$
q'_{t+1}(S_t, A_t) \leftarrow q'_t(S_t, A_t) + \alpha_t \left[ R_{t+1} + \gamma \max_{a'} q_t(S_{t+1}, a') - q'_t(S_t, A_t) \right]
$$

In SARSA, Q-Learning, and Double Q-Learning, the learning rate ($\alpha$) affects how much the model relies on the new Q-value from the recent state during updates.
- If the learning rate is too high, the model may become unstable due to high variance in updates.
- If the learning rate is too low, the model may converge slowly, but it will be more stable.

- *SARSA*

<p align = "center">
    <img src="../images/SARSAlr_graph.png" alt="Alt text" width="800"/>
</p>

In the early stage, the black line (lower learning rate) seem to learn more stably and accumulates higher rewards. But, the blue line (higher learning rate) shows lower rewards. This may be because the high learning rate model tries to converge quickly, but in the early stage, it hasn’t found the optimal policy yet.

In the later stage (around episode 3000), the blue line starts to increase rapidly. This could be due to a higher exploitation rate at that point—if the model has found a good policy, the high learning rate allows it to quickly update and reinforce recent Q-values, leading to faster improvement in rewards.

- *Q_Learning*
<p align = "center">
    <img src="../images/Q_Learnlr_graph.png" alt="Alt text" width="800"/>
</p>

The reward graph of Q-Learning behaves similarly to the SARSA graph. In the initial episodes, the model with a higher learning rate accumulates less reward. But, after around episode 3000, the model with the higher learning rate has a quick increase in accumulated reward, although it remains somewhat unstable.

- *Double_Q_Learning*
<p align = "center">
    <img src="../images/DQ_Learnlr_graph.png" alt="Alt text" width="800"/>
</p>

In the Double Q Learning graph, both models seem to have similar characteristics throughout the episodes. This may be because Double Q Learning uses two separate Q-value estimators so it less sensitive to high learning rate, which helps prevent the overestimation of Q-values and reduces the likelihood of overriding past values too aggressively.

##### Epsilon Decay
In this experiment we use 2 value of epsilon decay (0.9988 and 0.9982)



<p align = "center">
    <img src="../images/epsilon_decay.png" alt="Alt text" width="800"/>
</p>

- *Monte Carlo*

<p align = "center">
    <img src="../images/MCed.png" alt="Alt text" width="800"/>
</p>

<p align = "center">
    <img src="../images/MCed_graph.png" alt="Alt text" width="800"/>
</p>

- *SARSA*
<p align = "center">
    <img src="../images/SARSAed.png" alt="Alt text" width="800"/>
</p>

<p align = "center">
    <img src="../images/SARSAed_graph.png" alt="Alt text" width="800"/>
</p>


- *Q_Learning*

<p align = "center">
    <img src="../images/Q_Learned.png" alt="Alt text" width="800"/>
</p>

<p align = "center">
    <img src="../images/Q_Learned_graph.png" alt="Alt text" width="800"/>
</p>

- *Double_Q_Learning*

<p align = "center">
    <img src="../images/DQ_Learned.png" alt="Alt text" width="800"/>
</p>

<p align = "center">
    <img src="../images/DQ_Learned_graph.png" alt="Alt text" width="800"/>
</p>

In Double Q Learning, since it uses two separate policies, a lower epsilon decay (exploiting faster) may cause the model to not explore enough and try to exploit the policies it has. This is evident as the performance line appears to be lower than the one with a higher epsilon decay rate.

Graphs from all algorithms (except Monte Carlo) behave similarly. When the epsilon-decay is low (the model exploits faster), the accumulated Q-values are high near the center of the cart and pole positions. However, with a high epsilon-decay rate (the model explores longer), the accumulated Q-values become more uniform, and the graph appears flatter, as can see on the right side.

##### Discount factor

In this experiment we use 2 value of discount factor (0.1 and 0.5)

Monte Carlo :

$\begin{equation}
q(S_t, A_t) \leftarrow q(S_t, A_t) + \frac{1}{N(S_t,A_t)} (G_t - q(S_t, A_t))
\end{equation}$

$\begin{equation}
G_t = R_{t+1} + \gamma R_{t+2} + ... + \gamma^{T-t-1}R_T
\end{equation}$

Discount factor effect how much future reward value compare to current reward

SARSA : 
$$q_{t+1}(S_t, A_t) \leftarrow q_t(S_t, A_t) + \alpha_t \left[ R_{t+1} + \gamma q(S_{t+1}, A_{t+1}) - q_t(S_t, A_t) \right]
$$

Discount factor effect how much future action

Q_Learning :
$$
q_{t+1}(S_t, A_t) \leftarrow q_t(S_t, A_t) + \alpha_t \left[ R_{t+1} + \gamma \max_{a'} q_t(S_{t+1}, a') - q_t(S_t, A_t) \right]
$$

Double_Q_Learning :
$$
q_{t+1}(S_t, A_t) \leftarrow q_t(S_t, A_t) + \alpha_t \left[ R_{t+1} + \gamma \max_{a'} q'_t(S_{t+1}, a') - q_t(S_t, A_t) \right]
$$

$$
q'_{t+1}(S_t, A_t) \leftarrow q'_t(S_t, A_t) + \alpha_t \left[ R_{t+1} + \gamma \max_{a'} q_t(S_{t+1}, a') - q'_t(S_t, A_t) \right]
$$

In Q Learning and Double Q Learning dicount factor have similar effect to the value of max $q_t$ but in Double q it may less sensitive for high discount factor value from seperate two q-value

- Monte Carlo

<p align = "center">
    <img src="../images/MCdf_graph.png" alt="Alt text" width="800"/>
</p>

- SARSA

<p align = "center">
    <img src="../images/SARSAdf_graph.png" alt="Alt text" width="800"/>
</p>

- Q_Learning

<p align = "center">
    <img src="../images/Q_Learndf_graph.png" alt="Alt text" width="800"/>
</p>

- Double_Q_Learning

<p align = "center">
    <img src="../images/DQ_Learndf_graph.png" alt="Alt text" width="800"/>
</p>

#### Number of action
In this experiment we use 2 value of number of action (3 and 7)

<p align = "center">
    <img src="../images/number_action.png" alt="Alt text" width="800"/>
</p>

as see from the graph agent have different number of action to choose for interact with environment.

- Monte Carlo

<p align = "center">
    <img src="../images/MCac_graph.png" alt="Alt text" width="800"/>
</p>

- SARSA

<p align = "center">
    <img src="../images/SARSAac_graph.png" alt="Alt text" width="800"/>
</p>

- Q Learning

<p align = "center">
    <img src="../images/Q_Learnac_graph.png" alt="Alt text" width="800"/>
</p>

- Double Q Learning

<p align = "center">
    <img src="../images/DQ_Learnac_graph.png" alt="Alt text" width="800"/>
</p>

In this experiment, we didn't see trend of graph that are the significant information

#### Discrete state weight (Pole position)
In this experiment we use 2 value of discrete state weight focused on pole position (5 and 11)

- Monte Carlo

<p align = "center">
    <img src="../images/MCst.png" alt="Alt text" width="800"/>
</p>

- SARSA

<p align = "center">
    <img src="../images/SARSAst.png" alt="Alt text" width="800"/>
</p>

- Double_Q_Learning

<p align = "center">
    <img src="../images/DQ_Learnst.png" alt="Alt text" width="800"/>
</p>

- Q_Learning
<p align = "center">
    <img src="../images/Q_Learnst.png" alt="Alt text" width="800"/>
</p>

Discrete state weight is a parameter that indicates how many discrete state ranges when converting from a continuous to a discrete. A higher value means allowing the RL agent to learn more precise by distinguishing subtle differences in the environment.

### Q table


A Q-value is a number that tells the agent how good it is to take a certain action in a given state.

$$Q(S, A) = Expected future reward starting from state s and taking action A$$

In this case of `CartPole` it mean if the cart is on state S_t and has 3 actions, agent will looking at q table (Q(S_t, A_n) ; for all n) and looking for action that give the highest q_value, then agent will select the best action to response each state.

The below graph will show how q_values changing each algorithm when adjusting some hyperparameter
- 3D Q_values graph
    1. On the vertical axis represent how q_values are in each state
    2. On horizonral axis represent Cart position and Pole position 
        - In the center of both position (cart and pole) represent range of position (min - max) they allow without terminate 

So, in this graph can visualize how the characteristic trend for each algorithm will be. In the best case graph should shape like a peak of mountain that can show agent can learn to stabilize a pole to upright position and cart position didn't change to much.

<p align = "center">
    <img src="../images/Comparison_table.png" alt="Alt text" width="800"/>
</p>

In this image vertical show how hyper-parameter adjust, we use "Base Case" as a ground tooth of experiment and adjust parameter to see how it effect to the algorithm. Almost the hyper-parameter in base case are set as high value and in the adjusting part are decrease the parameter's value except the "Learning rate" that "Base Case" is lower than "Learning rate" configuration. 

**Configuration of "Base Case" and adjusting each hyper-parameter**
- **Action:** 5 | 7(Base)
- **State:** 5 | 11(Base)
- **Learning Rate:** 0.1(Base) | 0.5
- **Discount Factor:** 0.9982 | 0.9988(Base) 
- **Epsilon Decay:** 0.1 | 05(Base)

From the above comparision table, we can see which state has higher q value and which state has lower value or negative value, which mean higher value(Yellow) cartpole should stabilize better than the lower value(Blue).

### Observation state

The graph below is represent of the state of the agent (position and velocity of cart and pole)
- X axis: Time step
- Y axis: Values


This table represent label of the graph below 

|||
|--|--|
|Position cart|Position pole|
|Velocity cart|Velocity pole|

Note 
- Negative value: Left, Positive value: Right
- Cart's position range: -3 to 3 
- Pole's position range: ~ -0.42 to 0.42 radians

Criterion for best model
1. Longest alive time
2. Has stable cart and pole position

#### Monte Carlo

<p align = "center">
    <img src="../images/MC.png" alt="Alt text" width="1000"/>
</p>

On Monte Carlo can alive only 30 steps for all configuration, it mean this algorithm can not do this task (lower reward, lower q value) look at the 3D Q_value graph, we can see the q value graph will be see as negative or zero value, but we will choose the best configuration from the longest alive time, the best configuration is MC with has less action than 'base_case'

#### SARSA


<p align = "center">
    <img src="../images/SARSA.png" alt="Alt text" width="1000"/>
</p>

On SARSA algorithm has alive time more than MC, looking at the pole's position mostly configuration were terminated by pole's position out of bound(more than +-0.4) except 'Learning rate(Purple)' was terminated by cart out of bound(more than +- 3) and we choose the learning rate configuration as the best from alive time.

#### Q Learning

<p align = "center">
    <img src="../images/Q.png" alt="Alt text" width="1000"/>
</p>

Mostly of configuration on this algorithm agent will slide to left and then try to stabilize and sliding to right side, we can see that on the purple line has the longest alive time, so, we use the 'Learning rate' configuration as the best.

#### Double Q Learning

<p align = "center">
    <img src="../images/2Q.png" alt="Alt text" width="1000"/>
</p>

This algorithm look has the same as the above algorithm, so, we choose the 'Learning Rate' configuration for the best config.

### The Best algorithm


<p align = "center">
    <img src="../images/best.png" alt="Alt text" width="1000"/>
</p>

After we select the best configuration from each algorithm, we try find which algorithm is the best by looking at the highest alive time and how stable of pole position, we can see the orange line has the longest alive time and focus on the pole's position that position of this config has stable osilate for a long time, it also mean it can do their task well but it was terminated at the end of episode, but it still has the longest alive time for all model. 

So, for our experiment can tell that the best configuration of each algorithm is Q_learning with learning_rate = 0.5.

Although this is the best configuration that agent can stabilize but it cannot stay with the same cart's position it has a large amount of sliding, so, it will be the cause of terminate. In future improve, adding other eward term can be better example reward for sliding cart's position that if it has less sliding it will get reward or focus on velocity of pole if pole's velocity has suddenly change it will be penalty term, so, this addition should be improve the agent for stabilize task.